In [1]:
!pip install -q pyspark findspark mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 11.6 MB/s eta 0:00:00


In [97]:
import os

os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
!ls /usr/lib/jvm/java-11-openjdk-amd64

bin  conf  docs  include  jmods  legal	lib  man  release


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, expr
from pyspark.sql.types import StructType, StructField, StringType, DoubleType ,ArrayType ,LongType ,IntegerType
import mysql.connector
import os 
from configparser import ConfigParser
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode
from pyspark.sql import Row
import json

In [4]:
spark = SparkSession.builder \
    .appName("Data_Pipeline") \
    .getOrCreate()

In [6]:

headers = {
  'content-type': "application/json"
}
body = json.dumps({})
RestApiRequestRow = Row("verb", "url", "headers", "body")

api_url = "https://api.openweathermap.org/data/2.5/onecall?lat=14.497401&lon=-14.452362&exclude=hourly,daily&appid=101098fb7b42c64a657c60649632e063"
request_df = spark.createDataFrame([
  RestApiRequestRow("get", api_url, headers, body)
])



+----+--------------------+--------------------+----+
|verb|                 url|             headers|body|
+----+--------------------+--------------------+----+
| get|https://api.openw...|{content-type -> ...|  {}|
+----+--------------------+--------------------+----+



In [75]:
schema = StructType([

  
    StructField("current", StructType([
        StructField("dt", LongType(), True),

        StructField("temp", DoubleType(), True),

        StructField("humidity", IntegerType(), True),

        StructField("visibility", IntegerType(), True),
        StructField("wind_speed", DoubleType(), True),

    ]), True)
])

In [76]:
def executeRestApi(verb, url, headers, body):
  headers = {
      'content-type': "application/json"
  }
  res = None
  # Make API request, get response object back, create dataframe from above schema.
  try:
    if verb == "get":
      res = requests.get(url, data=body, headers=headers)
    else:
      res = requests.post(url, data=body, headers=headers)
  except Exception as e:
    return e
  if res != None and res.status_code == 200:
    return json.loads(res.text)
  return None

In [77]:
udf_executeRestApi = udf(executeRestApi, schema)

In [78]:
result_df = request_df \
             .withColumn("data", udf_executeRestApi(col("verb"), col("url"), col("headers"), col("body")))


In [101]:

from pyspark.sql.functions import split
from pyspark.sql.functions import from_json ,to_json
from pyspark.sql.types import MapType,StringType

current_schema=StructType([
StructField("dt", LongType(), True),

        StructField("temp", DoubleType(), True),

        StructField("humidity", IntegerType(), True),

        StructField("wind_speed", DoubleType(), True),
])

result_df_data = request_df.withColumn("data", udf_executeRestApi(col("verb"), col("url"), col("headers"), col("body")))\
.withColumn("data",to_json(col("data")))\
.withColumn("data", from_json("data", schema))\
.select( col('data.*'))\
.withColumn("current",to_json(col("current")))\
.withColumn("current", from_json("current", current_schema))\
.select( col('current.*'))\
.withColumnRenamed('dt', 'timestamp')\
.withColumnRenamed('temp', 'temperature')\
.show(truncate=False)


+----------+-----------+--------+----------+
|timestamp |temperature|humidity|wind_speed|
+----------+-----------+--------+----------+
|1685374190|315.24     |19      |3.21      |
+----------+-----------+--------+----------+



In [96]:


result_df_data.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://172.106.0.54:19320/meteodb") \
  .option("dbtable", "employee") \
  .option("user", "admin") \
  .option("password", "ixJQgvsI") \
  .save()

Py4JJavaError: ignored